In [2]:
import re 
import sys 
import numpy as np 
sys.path.append('/home/work/yuna/VLMEval') 
from dataset.dataloader import PostProcessor # contractions, numbers, punctuations
from analysis.pilot.analysis import vqa_score, str_to_list 
from analysis.utils.question_type_mapper import question_type 
from typing import List, Any
import pandas as pd 
from glob import glob 
import csv
import json
import os
from sentence_transformers import SentenceTransformer
import numpy as np 
from utils import answer_similarity, contains_korean
    
encoder = SentenceTransformer("all-MiniLM-L6-v2")
postprocessor = PostProcessor()   

In [12]:
qs = [] 
for f in glob('/home/work/yuna/VLMEval/analysis/pilot/questions/blind*.csv'): # Question paths 
    df = pd.read_csv(f) 
    # print(f, len(df.columns))
    qs.append(df) 
    
qs = pd.concat(qs)
qs = qs[['question_id', 'question', 'mean_acc']] # drop everything except question_id and question and mean_acc  
qs = pd.merge(qs, adf, on=['question_id'], how='left')  

In [7]:
def process_vqa(row):
    qid = int(row.get("question_id", "").strip()) 
    ans = get_answer(adf, qid) 
    processed_output = postprocessor.postprocess_answer(row.get("output", "")) 
    acc = vqa_score(processed_output, ans) 
    score = answer_similarity(processed_output, ans)
    item = {
            "qid": row.get("question_id", "").strip(),
            "question": row.get("question", "").strip(),  
            "question_type": question_type[int(row.get("question_id", "").strip())],
            "model": row.get("model", "").strip(),
            "answer": ans, 
            # "filename": row.get("filename", "").strip(),
            "raw_output": row.get("output", "").strip(),
            "processed_output": processed_output, # row.get("processed_output", "").strip(),
            "acc": acc,
            'score': float(score) 
            }
    # print(item) 
    return item 

In [ ]:
for file in glob('/home/work/yuna/VLMEval/lm_vqa/*'):  
    convert_results(file, "LLM") 

In [38]:
def read_human_results(folder='./responses'): 
    dfs=[]
    for f in glob(f'{folder}/*.xlsx'): 
        print(f) 
        dfs.append(pd.read_excel(f)) 
    df = pd.concat(dfs)
    df = df.melt(id_vars=['Timestamp', "이름 또는 이니셜 Name or Initials ", 'Score']).rename(columns={'variable': 'question', 'value': 'answer', "이름 또는 이니셜 Name or Initials ": "subj"})
    # df['question'] = df['question'].str.split('\n').str[0]
    df[['question', 'question (Korean)']] = df['question'].str.split('\n', expand=True)[[0,1]] 
    df['answer'] = df['answer'].replace({'Yes 네': 'yes', "No 아니오": "no"}) 

    return df 

In [77]:
hm = read_human_results("/home/work/yuna/VLMEval/analysis/pilot/responses") 
# hm = pd.merge(hm, adf, on=['question'], how='left') 
hm['question'] = hm['question'].str.replace('scene', 'photo')

/home/work/yuna/VLMEval/analysis/pilot/responses/VQA-SPUB Pilot Experiment (Responses) (1).xlsx
/home/work/yuna/VLMEval/analysis/pilot/responses/VQA-SPUB Pilot Experiment Pt.2 (Responses) (1).xlsx
/home/work/yuna/VLMEval/analysis/pilot/responses/VQA-SPUB Pilot Experiment Pt.3 (Responses).xlsx


In [79]:
len(allqs)

81573

In [78]:
hm = pd.merge(hm, allqs, on=['question'], how='left') 
hm['answer'] = hm['answer'].replace({'Yes 네': 'yes', "No 아니오": "no"}) 
mask = hm['question_id'].isna()  
hm[mask].question.unique()

array(['성별', '연락처 (필수 아님)', 'Name or Initials 이름 또는 이니셜 '], dtype=object)

In [80]:
len(hm)

10255

In [81]:
hm['subj'] = hm['subj'].fillna(hm['Timestamp']) 

In [48]:
import re 
    
korean_translations = {} 
kr = pd.concat([pd.read_csv('/home/work/yuna/VLMEval/analysis/pilot/responses/korean_answers_translated.csv'),
                pd.read_csv('/home/work/yuna/VLMEval/analysis/pilot/responses/korean_answers_translated (2).csv')]) 

for i, r in kr.iterrows(): 
    kr_ans = r['korean'] 
    kr_ans = postprocessor.postprocess_answer(kr_ans)  
    en_ans = r['answer'] 
    if kr_ans not in korean_translations.keys(): 
        korean_translations[kr_ans] = en_ans 
len(korean_translations)

183

In [97]:
pd.DataFrame(need_tran).to_csv('/home/work/yuna/hpa/eda/need_translate.csv')

In [104]:
tr = pd.read_csv('./translate.csv')
tr = tr.drop(columns=['Unnamed: 0'])

In [107]:
first_col = tr.columns[0]  # e.g., 'question'
second_col = tr.columns[1]  # e.g., 'subj' or 'answer'
dictionary = dict(zip(tr[first_col].astype(str), tr[second_col].astype(str))) 

In [ ]:
korean_translations.update(dictionary)
korean_translations 

In [83]:
df = hm.dropna(subset=['question_id'], axis=0)  
len(df)

10150

In [112]:
# Function to save dictionary to Python file
# def save_dict_to_python_file(dictionary, filename):
with open('./korean_translations.py', 'w', encoding='utf-8') as f:
    f.write('korean_translations = ')
    json.dump(korean_translations, f, ensure_ascii=False, indent=4)
    f.write('\n')

In [111]:
output_folder = f"/home/work/yuna/hpa/results/humans_processed" 
os.makedirs(output_folder, exist_ok=True) 
need_tran=[] 

for subj in df.subj.unique(): 
    tmp = df[df['subj'] == subj] 
    filename = os.path.join(output_folder, f'{subj}.jsonl' ) 

    with open(filename, "w", encoding="utf-8") as fout: 
        for i, row in tmp.iterrows():  
            # print(row)
            qid = row['question_id'] 
            # print(int(qid) )
            output = row.get("answer", "") 
            processed_output = postprocessor.postprocess_answer(str(output)) 
            ans = get_answer(adf, qid) 
            
            if contains_korean(processed_output): 
                if processed_output in korean_translations.keys(): 
                    processed_output = korean_translations[processed_output] 
                else: 
                    print(row.get("question", "").strip(), processed_output) 
                    if processed_output not in need_tran:
                        need_tran.append(processed_output)
            acc = vqa_score(processed_output, ans) 
            score = float(answer_similarity(processed_output, ans))
            
            item = {
                    "qid": qid,
                    "question": row.get("question", "").strip(),  
                    "question_type": question_type[qid],
                    "subj": str(subj), 
                    # "answer": ans, 
                    "raw_output": output, 
                    "processed_output": processed_output, 
                    "acc": acc ,
                    'score': score 
                    }
            fout.write(json.dumps(item, ensure_ascii=False) + "\n")
                    
    print(f"✅ Saved JSONL to: {filename}") 

✅ Saved JSONL to: /home/work/yuna/hpa/results/humans_processed/성민석.jsonl
✅ Saved JSONL to: /home/work/yuna/hpa/results/humans_processed/Kathy.jsonl
✅ Saved JSONL to: /home/work/yuna/hpa/results/humans_processed/Winnie.jsonl
✅ Saved JSONL to: /home/work/yuna/hpa/results/humans_processed/안덕현.jsonl
✅ Saved JSONL to: /home/work/yuna/hpa/results/humans_processed/성희.jsonl
✅ Saved JSONL to: /home/work/yuna/hpa/results/humans_processed/박지윤.jsonl
✅ Saved JSONL to: /home/work/yuna/hpa/results/humans_processed/박정원.jsonl
✅ Saved JSONL to: /home/work/yuna/hpa/results/humans_processed/Cm.jsonl
✅ Saved JSONL to: /home/work/yuna/hpa/results/humans_processed/박호조.jsonl
✅ Saved JSONL to: /home/work/yuna/hpa/results/humans_processed/Dowon Hwang.jsonl
✅ Saved JSONL to: /home/work/yuna/hpa/results/humans_processed/조희승.jsonl
✅ Saved JSONL to: /home/work/yuna/hpa/results/humans_processed/EJ.jsonl
✅ Saved JSONL to: /home/work/yuna/hpa/results/humans_processed/BJH.jsonl
✅ Saved JSONL to: /home/work/yuna/hpa/res